In [1]:
# install main library YFinance
!pip install yfinance

In [5]:
!pip install pandas_datareader

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date


# 1) Understanding Data-Driven Decisions data pulls

In [61]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=2000, month=1, day=1)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=22
Period for indexes: 2000-01-01 to 2024-04-22 


## A1

In [4]:
# Real Potential Gross Domestic Product (GDPPOT), Billions of Chained 2012 Dollars, QUARTERLY
# https://fred.stlouisfed.org/series/GDPPOT
gdppot = pdr.DataReader("GDPC1", "fred", start=start)

In [8]:
gdppot['gdppot_us_yoy'] = gdppot.GDPC1/gdppot.GDPC1.shift(4)-1
gdppot['gdppot_us_qoq'] = gdppot.GDPC1/gdppot.GDPC1.shift(1)-1
gdppot.tail(10)

,GDPC1,gdppot_us_yoy,gdppot_us_qoq
DATE,,,
2021-07-01,21483.083,0.047353,0.008144
2021-10-01,21847.602,0.054211,0.016968
2022-01-01,21738.871,0.035651,-0.004977
2022-04-01,21708.160,0.018706,-0.001413
2022-07-01,21851.134,0.017132,0.006586
2022-10-01,21989.981,0.006517,0.006354
2023-01-01,22112.329,0.017179,0.005564
2023-04-01,22225.350,0.023825,0.005111
2023-07-01,22490.692,0.029269,0.011939


In [71]:
gdppot["avg_yoy"] = gdppot.gdppot_us_yoy.rolling(4).mean()

In [72]:
gdppot.tail()

,GDPC1,gdppot_us_yoy,gdppot_us_qoq,avg_yoy
DATE,,,,
2022-10-01,21989.981,0.006517,0.006354,0.019501
2023-01-01,22112.329,0.017179,0.005564,0.014884
2023-04-01,22225.350,0.023825,0.005111,0.016163
2023-07-01,22490.692,0.029269,0.011939,0.019197
2023-10-01,22679.255,0.031345,0.008384,0.025404


## A2

In [75]:
# https://fred.stlouisfed.org/series/DGS1
dgs2 = pdr.DataReader("DGS2", "fred", start=start)
dgs2.tail()

,DGS2
DATE,
2024-04-12,4.88
2024-04-15,4.93
2024-04-16,4.97
2024-04-17,4.93
2024-04-18,4.98


In [76]:
# https://fred.stlouisfed.org/series/DGS5
dgs10 = pdr.DataReader("DGS10", "fred", start=start)
dgs10.tail()

,DGS10
DATE,
2024-04-12,4.50
2024-04-15,4.63
2024-04-16,4.67
2024-04-17,4.59
2024-04-18,4.64


In [77]:
result = dgs2.join(dgs10, how="inner")

In [78]:
result.head()

,DGS2,DGS10
DATE,,
2000-01-03,6.38,6.58
2000-01-04,6.30,6.49
2000-01-05,6.38,6.62
2000-01-06,6.35,6.57
2000-01-07,6.31,6.52


In [79]:
result["diff"]=result["DGS10"]-result["DGS2"]

In [44]:
result.dtypes

DGS2     float64
DGS10    float64
diff     float64
dtype: object

In [82]:
result.nsmallest(1, 'diff')

,DGS2,DGS10,diff
DATE,,,
2023-07-03,4.94,3.86,-1.08


## A3

In [246]:
# first method

gspc_daily = yf.download(tickers = "^GSPC",
                    start="2019-04-09",
                      end="2024-04-10",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [248]:
gspc_daily[(gspc_daily.index=='2024-04-09')]['Close'][0]/gspc_daily[(gspc_daily.index=='2019-04-09')]['Close'][0]

1.8101279426847174

In [249]:
mxx_daily = yf.download(tickers = "^MXX",
                    start="2019-04-09",
                      end="2024-04-10",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [250]:
mxx_daily[(mxx_daily.index=='2024-04-09')]['Close'][0]/mxx_daily[(mxx_daily.index=='2019-04-09')]['Close'][0]

1.2750624912566744

In [251]:
# second method
# indexes: https://stooq.com/t/
spx_index = pdr.get_data_stooq('^SPX', start, end)
spx_index.head()

,Open,High,Low,Close,Volume
Date,,,,,
2024-04-22,4987.33,5038.84,4969.40,5010.60,2.352809e+09
2024-04-19,5005.44,5019.02,4953.56,4967.23,2.735280e+09
2024-04-18,5031.52,5056.66,5001.89,5011.12,2.217369e+09
2024-04-17,5068.97,5077.96,5007.25,5022.21,2.295628e+09
2024-04-16,5064.59,5079.84,5039.83,5051.41,2.347350e+09


In [85]:
# 252 trading days a year
  # REVERSE ORDER OF THE DATA!!!
spx_index['spx_dod'] = (spx_index.Close/spx_index.Close.shift(-1)-1)
spx_index['spx_qoq'] = (spx_index.Close/spx_index.Close.shift(-63)-1)
spx_index['spx_yoy'] = (spx_index.Close/spx_index.Close.shift(-252)-1)

In [133]:
# 252 trading days a year
  # REVERSE ORDER OF THE DATA!!!
ipc_index['ipc_dod'] = (ipc_index.Close/ipc_index.Close.shift(-1)-1)
ipc_index['ipc_qoq'] = (ipc_index.Close/ipc_index.Close.shift(-63)-1)
ipc_index['ipc_yoy'] = (ipc_index.Close/ipc_index.Close.shift(-252)-1)

In [86]:
spx_index.head()

,Open,High,Low,Close,Volume,spx_dod,spx_qoq,spx_yoy
Date,,,,,,,,
2024-04-19,5005.44,5019.02,4953.56,4967.23,2.727319e+09,-0.008759,0.026327,0.195621
2024-04-18,5031.52,5056.66,5001.89,5011.12,2.217369e+09,-0.002208,0.048145,0.206083
2024-04-17,5068.97,5077.96,5007.25,5022.21,2.295628e+09,-0.005781,0.059715,0.209786
2024-04-16,5064.59,5079.84,5039.83,5051.41,2.347350e+09,-0.002057,0.059889,0.220843
2024-04-15,5149.67,5168.43,5052.47,5061.82,2.319452e+09,-0.012021,0.058110,0.220828


In [134]:
ipc_index.head()

,Open,High,Low,Close,Volume,ipc_dod,ipc_qoq,ipc_yoy
Date,,,,,,,,
2024-04-19,55811.19,56036.72,55579.69,55862.85,NaN,0.002214,0.021040,0.028629
2024-04-18,55597.79,55941.62,55585.82,55739.43,208671898.0,0.005842,0.011154,0.024895
2024-04-17,55788.74,56090.32,55383.25,55415.69,244258908.0,-0.006838,-0.001539,0.011624
2024-04-16,56009.73,56148.00,55626.75,55797.25,225862911.0,-0.003336,0.003420,0.024554
2024-04-15,56536.47,56709.12,55924.10,55984.00,170645737.0,-0.010287,0.009835,0.023552


In [89]:
spx_truncated = spx_index[(spx_index.index>='2019-04-09') & (spx_index.index<='2024-04-09')]

In [121]:
spx_start=spx_index[(spx_index.index=='2019-04-09')]['Close']

In [122]:
spx_end=spx_index[(spx_index.index=='2024-04-09')]['Close']

In [135]:
ipc_start=ipc_index[(ipc_index.index=='2019-04-09')]['Close']

In [136]:
ipc_end=ipc_index[(ipc_index.index=='2024-04-09')]['Close']

In [125]:
spx_end.iloc[0]/spx_start.iloc[0]

1.8101278576888333

In [137]:
ipc_end.iloc[0]/ipc_start.iloc[0]

1.2750627164512114

## A4

In [157]:
# INDEXES from Yahoo Finance
# DAX index (XETRA - XETRA Delayed Price. Currency in EUR)
# WEB: https://finance.yahoo.com/quote/%5EGDAXI
jpm_daily = yf.download(tickers = "JPM",
                     start="2023-01-01",
                      end="2023-12-31",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [158]:
goog_daily = yf.download(tickers = "GOOG",
                    start="2023-01-01",
                      end="2023-12-31",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [159]:
msft_daily = yf.download(tickers = "MSFT",
                    start="2023-01-01",
                      end="2023-12-31",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [160]:
aapl_daily = yf.download(tickers = "AAPL",
                    start="2023-01-01",
                      end="2023-12-31",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [161]:
brkb_daily = yf.download(tickers = "BRK-B",
                    start="2023-01-01",
                      end="2023-12-31",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [162]:
sr2222_daily = yf.download(tickers = "2222.SR",
                    start="2023-01-01",
                      end="2023-12-31",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [172]:
(jpm_daily['Adj Close'].max()-jpm_daily['Adj Close'].min())/jpm_daily['Adj Close'].max()

0.28249927707093897

In [173]:
(goog_daily['Adj Close'].max()-goog_daily['Adj Close'].min())/goog_daily['Adj Close'].max()

0.3924520921912013

In [174]:
(msft_daily['Adj Close'].max()-msft_daily['Adj Close'].min())/msft_daily['Adj Close'].max()

0.4242066914981641

In [175]:
(brkb_daily['Adj Close'].max()-brkb_daily['Adj Close'].min())/brkb_daily['Adj Close'].max()

0.20775750091289963

In [176]:
(sr2222_daily['Adj Close'].max()-sr2222_daily['Adj Close'].min())/sr2222_daily['Adj Close'].max()

0.21393070882746015

## A5

In [214]:
# get actions, incl. dividends - as a dataFrame
jpm = yf.Ticker('JPM')
jpm_df=jpm.get_actions()
jpm_div=jpm_df[(jpm_df.index >='2023-01-01') & (jpm_df.index <='2023-12-31')]['Dividends'].sum()

In [215]:
jpm_div/jpm_daily[jpm_daily.index==jpm_daily.index.max()]['Adj Close'][0]*100

2.409608190645897

In [216]:
goog = yf.Ticker('GOOG')
goog_df=goog.get_actions()
goog_div=goog_df[(goog_df.index >='2023-01-01') & (goog_df.index <='2023-12-31')]['Dividends'].sum()

In [217]:
goog_div/goog_daily[goog_daily.index==goog_daily.index.max()]['Adj Close'][0]*100

0.0

In [218]:
msft = yf.Ticker('MSFT')
msft_df=msft.get_actions()
msft_div=msft_df[(msft_df.index >='2023-01-01') & (msft_df.index <='2023-12-31')]['Dividends'].sum()

In [219]:
msft_div/msft_daily[msft_daily.index==msft_daily.index.max()]['Adj Close'][0]*100

0.7433143940964608

In [220]:
brkb = yf.Ticker('BRK-B')
brkb_df=brkb.get_actions()
brkb_div=brkb_df[(brkb_df.index >='2023-01-01') & (brkb_df.index <='2023-12-31')]['Dividends'].sum()

In [221]:
brkb_div/brkb_daily[brkb_daily.index==brkb_daily.index.max()]['Adj Close'][0]*100

0.0

In [222]:
sr2222 = yf.Ticker('2222.SR')
sr2222_df=sr2222.get_actions()
sr2222_div=sr2222_df[(sr2222_df.index >='2023-01-01') & (sr2222_df.index <='2023-12-31')]['Dividends'].sum()

In [224]:
sr2222_div/sr2222_daily[sr2222_daily.index==sr2222_daily.index.max()]['Adj Close'][0]*100

2.770149819503256